# Tic Tac Toe

Given rules of the game we want to use a version of genetic algorithm to train our model for playing ttt.

## The idea

- Any state of the board is represented by a vector in $\mathbb R^9$. Although it seems natural for the case at hand, we may keep in mind to investigate $\mathbb R^n$. 
    - What is this representation?
    - Is it a good idea to represent empty slots with 0's?
    - Should one use bitwise operators instead?
- Every player (strategy) is a set of weights, biases (matrices) $W$ and $b$. It takes a state as an input and decides where to make a move. The output is also $\mathbb R^9$.
- We start with just one layer, meaning that we have $$W = (9\times 9), ~~ \text{and} ~~ b=(9\times 1).$$
- At the very beginning (generation 0) we generate randomly a population of $N$ players, i.e. a set of pairs $(W_i,b_i)$, with $i=1,\dots,N$, and let them play against each other (there are $N(N-1)$ games in total).
- We score them somehow according to their performance in this competition. They get to reproduce according to their score. First $N_{best}$ cross-bread leaving offsprings. To complete new generation there are several options:
    1. We add $N_{rand}$ new (random) players.
    2. We randomly choose several lucky ones among the rest and let them reproduce as well.
    3. We cross-bread all with all and chose randomly a number of lucky offsprings.

## Implementation

### Importing necessary packages

In [1]:
%matplotlib inline
%matplotlib nbagg

import numpy as np
#import csv
from IPython.display import clear_output
from decimal import *

import matplotlib.pyplot as plt
#from sklearn.neural_network import MLPClassifier
#from sklearn.model_selection import train_test_split

plt.rcParams["figure.figsize"] = (8, 8)
plt.rcParams["font.size"] = 14

### Definitions
All classes and functions taking field as an argument consider it as $(9\times1)$ vector (not $(3\times3)$ matrix)

In [75]:
def sigmoid(x):
    return 1/(1+np.exp(-x))
       
class strategy: # Players knowing that going in the occupied slot is forbidden
    def __init__(self,weights,biases,mutation_rate=0,name=None): # Is mutation rate just learning rate?
        self.weights=weights
        self.biases=biases
        self.name=name
        self.mutation_rate=mutation_rate
        
    def intensity(self,field): # Given the state of field it computes probability for every move using weights and biases
        n_layers=len(self.biases)
        x_in=field
        for counter in range(n_layers):
            argument=np.matmul(self.weights[counter],x_in)+self.biases[counter]
            x_in=sigmoid(argument)
        return argument

    
    def occupied_q(self,field,slot): # Checks is a particular slot is occupied
        return field[slot]
    
    def whereto(self,field,one_hot=True): # Decides where to make the next move by finding the maximal intensity among unoccupied slots
        sorted_args=np.argsort(self.intensity(field))[::-1]
        number=0
        while self.occupied_q(field,sorted_args[number]):
            number=number+1
            
        if not one_hot:
            return sorted_args[number]
        else:
            return np.eye(9)[sorted_args[number]]

class history: # Used for keeping track of evolution
    def __init__(self,names_all=np.array([]),names_best=np.array([]),scores_all=np.array([]),scores_best=np.array([])):
        #self.weights=weights
        #self.biases=biases
        self.scores_all=scores_all
        self.scores_best=scores_best
        self.names_all=names_all
        self.names_best=names_best
        #self.generation=generation
        
def winner_q(field):
    
    reward=np.array([2,-1]) # 2 points for winning and -1 point for losing
    field=field.reshape((3,3))
    plus=np.array([1,1,1])
    minus=np.array([-1,-1,-1])
    def indicator(vec):
        return ((field[0]==vec).all() or (field[1]==vec).all() or (field[2]==vec).all() 
            or (field.T[0]==vec).all() or (field.T[1]==vec).all() or (field.T[2]==vec).all() 
            or (np.diag(field)==vec).all() or (np.diag(np.fliplr(field))==vec).all())    
    if indicator(plus):
        return reward
    elif indicator(minus):
        return reward[::-1]
    else:
        return np.array([0,0])
    
def game(strat1,strat2,verbose=False):
    
    if verbose:
        field=np.zeros(9)
        counter=0
        while (not (winner_q(field)).any()) and counter<10:
            print('Step 1: \n')
            print(strat1.intensity(field))
            field=field+strat1.whereto(field)
            counter=counter+1
            print(field.reshape((3,3)))
            if (winner_q(field)).any() or counter>8:
                break
            print(strat2.intensity(field))
            field=field-strat2.whereto(field)
            counter=counter+1
            print(field.reshape((3,3)))
    else:
        field=np.zeros(9)
        counter=0
        while (not (winner_q(field)).any()) and counter<10:
            field=field+strat1.whereto(field)
            counter=counter+1
            if (winner_q(field)).any() or counter>8:
                break
            field=field-strat2.whereto(field)
            counter=counter+1
#         if winner_q(field)[0]>0:
#             print('Player',strat1.name,'wins against',strat2.name)
#         elif winner_q(field)[0]<0:
#             print('Player',strat1.name,'loses against',strat2.name)
#         else:
#             print('It is a tie in:',strat1.name,'vs.',strat2.name)
        return winner_q(field)


def offspring(strat1,strat2,rel_score):
    
    w1=strat1.weights
    b1=strat1.biases
    name1=strat1.name
    mr1=strat1.mutation_rate
    
    shape_w=w1.shape
    shape_b=b1.shape
    
    noise1_w=(2*np.random.random(shape_w)-1)*mr1
    noise1_b=(2*np.random.random(shape_b)-1)*mr1
    frac1=rel_score/(1+rel_score)
    
    w2=strat2.weights
    b2=strat2.biases
    name2=strat2.name
    mr2=strat2.mutation_rate
    
    noise2_w=(2*np.random.random(shape_w)-1)*mr2
    noise2_b=(2*np.random.random(shape_b)-1)*mr2
    frac2=1-frac1
    
    new_weights=w1*(1+noise1_w)*frac1+w2*(1+noise2_w)*frac2
    new_biases=b1*(1+noise1_b)*frac1+b2*(1+noise2_b)*frac2
    new_mr=mr1*frac1+mr2*frac2
#     new_name='('+name1+','+name2+')'
    
    
    if rel_score>=1:
        new_name=strat1.name
    else:
        new_name=strat2.name
    
    return strategy(weights=new_weights,biases=new_biases,name=new_name,mutation_rate=new_mr)



def mutation_const_rate(strat,indx):
    
    w=strat.weights
    b=strat.biases
    mr=strat.mutation_rate
    noise_w=(2*np.random.random(w.shape)-1)*mr
    noise_b=(2*np.random.random(b.shape)-1)*mr
    
    new_weights=w*(1+noise_w)
    new_biases=b*(1+noise_b)
    new_mr=mr
    new_name=str(indx)
    
    return strategy(weights=new_weights,biases=new_biases,name=new_name,mutation_rate=new_mr)




def get_names(players):
    names=np.array([])
    for player in players:
        names=np.append(names,player.name)
    return names


def tournament(players):
    scores=np.zeros(num_players) # Initialize scores with zeros
    for counter1 in range(num_players):
        for counter2 in range(num_players):
            match=game(players[counter1],players[counter2])
            scores[counter1]+=match[0]
            scores[counter2]+=match[1]
#             clear_output(wait=True)
#             print('Games played: %',int(10**4*(counter1*num_players+counter2+1)/num_players**2)/100)
#             print('Players: ',players[counter1].name,players[counter2].name)
#             print(scores[players[counter1].name])
#             print(scores[players[counter2].name],'\n')
    return scores


def new_gen_random(players): # Returns new batch of players and scores of the current generation
    names=get_names(players)
    print(names)
    scores=tournament(players)
    print(scores)
    scores_dict=np.stack((names,scores))
    
    ind_sorted=np.argsort(scores)
    best_performers=players[ind_sorted[num_players-num_best::]][::-1] # Descedning order
    
    names_best=get_names(best_performers) # Descending order
    #print(names_best)
    scores_best=scores[ind_sorted[num_players-num_best::]][::-1] #Descending order
    #print(scores_best)
    best_scores_dict=np.stack((names_best,scores_best))
    
    
    elite_offsprings=np.array([])
    for counter1 in range(num_best):
        for counter2 in range(counter1,num_best):
            player1=best_performers[counter1]
            player2=best_performers[counter2]            
            ratio=scores_best[counter1]/scores_best[counter2]
            elite_offsprings=np.append(elite_offsprings,offspring(player1,player2,ratio))
    
    
    random_choice=np.array([strategy(weights=2*np.random.random((num_layers,9,9))-1,
                           biases=2*np.random.random((num_layers,9))-1,
                           mutation_rate=0,
                           name=str(i)) for i in range(num_players+num_random*generation,num_players+num_random*(generation+1))])
    return np.append(elite_offsprings,random_choice),scores_dict,best_performers,best_scores_dict



def new_gen_fromone(players): # Returns new batch of players and scores of the current generation
    
    scores=tournament(players)
    #print(scores)
    index=np.argmax(scores)
    
    parent=players[index]
        
    return np.append(parent,np.array([mutation_const_rate(parent,index) for _ in range(num_players-1)])),scores

### Generate initial batch of players

In [250]:
np.random.seed(0)
num_best=3
num_random=3
num_players=np.int(num_best*(num_best+1)/2)+num_random
num_layers=2
generation=0

players=np.array([strategy(weights=2*np.random.random((num_layers,9,9))-1,
                           biases=2*np.random.random((num_layers,9))-1,
                           mutation_rate=0.3,
                           name=str(i)) for i in range(num_players)])
hist=history(names_all=get_names(players))

threshold=2*num_random
print(threshold)
print(num_players)

6
9


### Best players crossbreading + random

In [77]:
for counter in range(1):
    players,current_scores,best_strategies,best_scores=new_gen_random(players)
    generation=generation+1
    #hist.names_all=np.append(hist.names_all,get_names(players))
    #hist.scores_all=np.append(hist.scores_all,gen_score)
    #hist.names_best=np.append(hist.names_best,get_names(best_strategies))
    #hist.scores_best=np.append(hist.scores_best,best_scores)
    print(best_scores.T)
    print(get_names(players))
print(generation)

['0' '1' '2' '3' '4' '5' '6' '7' '8']
[13.  6. 17.  4.  8.  7. 10.  5.  1.]
[['2' '17.0']
 ['0' '13.0']
 ['6' '10.0']]
['2' '2' '2' '0' '0' '6' '9' '10' '11']
1


### One parent mutations

In [252]:
best0=players[0]

In [253]:
for _ in range(100):
    #print(get_names(players))
    players,scores=new_gen_fromone(players)
    #print(scores,'\n')
    generation+=1
print(generation)

101


In [249]:
field=np.array([[1,0,1],
                [1,-1,0],
                [-1,-1,0]])
print('Current best:')
print(field+players[0].whereto(field.reshape(-1)).reshape(3,-1),'\n')

#print('Initial best:')
#print(field+best0.whereto(field.reshape(-1)).reshape(3,-1))

Current best:
[[ 1.  0.  1.]
 [ 1. -1.  0.]
 [-1. -1.  1.]] 



In [27]:
field=np.array([[0,0,0],
                [-1,-1,0],
                [0,1,1]])
print(players[0].intensity(field.reshape(9)))
print(players[3].intensity(field.reshape(9)))
print(np.argmax(players[0].intensity(field.reshape(9))))
print(np.argmax(players[2].intensity(field.reshape(9))))
print(np.argmax(players[8].intensity(field.reshape(9))))

[ 0.84955189 -1.07961474  0.27725177  1.08092424  1.3713455  -0.37533411
  1.40599152 -1.115399   -0.25129812]
[ 0.94643846 -0.8386913   0.29870048  0.80525908  1.73803349 -0.39658534
  1.31436656 -1.56768852 -0.33094924]
6
6
4


In [11]:
field=np.array([[0,0,0],
                [-1,-1,0],
                [0,1,1]])
field+best_strategies[0].whereto(field.reshape(-1)).reshape(3,-1)

array([[ 0.,  0.,  0.],
       [-1., -1.,  1.],
       [ 0.,  1.,  1.]])

In [322]:
alpha=3

field=np.array([[1,0,0],
                [1,-1,-1],
                [-1,0,1]])

strat1=strategy(weights=w,biases=b)
strat2=strategy(weights=w+alpha*diff_w,biases=b+alpha*diff_b)

print(strat1.whereto(field.reshape(9),one_hot=False))
print(strat2.whereto(field.reshape(9),one_hot=False))

1
2
